## Algoritmo 
(se considera que los índices de los arrays inician en el valor $1$ y no en el $0$)

Entrada: $A \in \mathbb{R}^{n \times n}$.

Salida: factores $L, U \in \mathbb{R}^{n \times n}$, vector $piv \in \mathbb{R}^{n}$. El vector $piv$ se utiliza para construir a la matriz $P$.

Inicializar L con matriz identidad. Inicializar U como una matriz de ceros.

Para $j$ de $1$ a $n$ hacer:

Si $j$ es 1 definir $v = A(:,1)$ # v es la primer columna de $A$.

En otro caso:

Calcular $\hat{a} = P_{j-1} \cdots P_1 A(:,j)$ #aquí se utiliza al vector $piv$ para evitar hacer multiplicaciones de matriz vector, sólo se realizan intercambio de renglones

Resolver $L(1:j-1,1:j-1)z = \hat{a}(1:j-1)$ para $z \in \mathbb{R}^{j-1}$ por sustitución hacia delante

Definir $U(1:j-1,j) = z$

Definir $v(j:n) = \hat{a}(j:n) - L(j:n, 1:j-1)*z$

Determinar $\mu$ con $ j \leq \mu \leq n$ tal que $|v(\mu)| = ||v(j:n)||_\infty$

Definir $piv(j)=\mu$ # piv es un vector de tamaño $n-1$

$v(j) \leftrightarrow v(\mu)$ # $\leftrightarrow$ significa intercambiar $j$-ésima entrada de $v$ por $\mu$-ésima entrada de $v$.

$L(j,1:j-1) \leftrightarrow L(\mu,1:j-1)$ #$\leftrightarrow$ significa intercambio

Definir $U(j,j) = v(j)$.

Si $v(j) \neq 0 $ definir $L(j+1:n,j) = v(j+1:n)/v(j)$


In [3]:
import numpy as np

In [27]:
#AUXILIARES
def imprime_status(col_actual, P,L,U):
    print("\n===========================")
    print("ITERACION:  ", col_actual)
    print("Matriz L")
    print(L)
    print("Matriz U")
    print(U)
    print("Matriz P")
    print(P)
    
def intercambio_filas(M, a, b):
    fila_aux = M[a, :].copy()
    M[a, :] =  M[b, :]
    M[b, :] = fila_aux
    return M
    

In [55]:
def nuestro_lup(A):
    """
     Parameters
    ----------   
    A : ndarray
        2D array containing data with `float` type.
       
    Returns
    -------
    L
        ndarray of matrix L un LUP factorization
    U
        ndarray of matrix U un LUP factorization
    P
        ndarray of matrix P un LUP factorization
    """
    n = len(A)
    L = np.zeros((n,n), dtype=np.float)
    U = A.copy()
    P =  np.identity(n, dtype=np.float) 

    for col_actual in range(n):
        #imprime_status(col_actual, P,L,U)

        #Get maximum row to swap values
        row_mayor = col_actual
        a_hat = abs(U[col_actual,col_actual])
        for row in range(col_actual, n): 
            a = abs(U[row, col_actual])
            if a > a_hat : 
                row_mayor = row
                a_hat = a

        #Swap values
        U = intercambio_filas(U, row_mayor, col_actual)
        L = intercambio_filas(L, row_mayor, col_actual)
        P = intercambio_filas(P, row_mayor, col_actual) 


        for j in range(col_actual+1, n):
            vj = U[col_actual, col_actual]
            if (vj !=0):
                multi =  U[j, col_actual] / vj
                L[j, col_actual] =  multi
                U[j, :] = U[j, :] - multi * U[col_actual, :]

    L = L + np.identity(n, dtype=np.float)
    return L, U, P 


In [56]:
from scipy.linalg import lu

In [57]:

A = np.array([[ 1, 4,-2], \
              [-3, 9, 8],
              [ 5, 1,-6]], dtype=np.float)


L, U, P = nuestro_lup(A)
p, l, u = lu(A)

print("Matriz L")
print(L)
print("Matriz U")
print(U)
print("Matriz P")
print(P)


print("Matriz l")
print(l)
print("Matriz u")
print(u)
print("Matriz p")
print(p)

print("====== DIFERENCIA ========")
print("Matriz l")
print(l-L)
print("Matriz u")
print(u-U)
print("Matriz p")
print(p-P)

Matriz L
[[ 1.          0.          0.        ]
 [-0.6         1.          0.        ]
 [ 0.2         0.39583333  1.        ]]
Matriz U
[[ 5.          1.         -6.        ]
 [ 0.          9.6         4.4       ]
 [ 0.          0.         -2.54166667]]
Matriz P
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
Matriz l
[[ 1.          0.          0.        ]
 [-0.6         1.          0.        ]
 [ 0.2         0.39583333  1.        ]]
Matriz u
[[ 5.          1.         -6.        ]
 [ 0.          9.6         4.4       ]
 [ 0.          0.         -2.54166667]]
Matriz p
[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
====== DIFERENCIA ========
Matriz l
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.11022302e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
Matriz u
[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00 -8.8817842e-16]
 [ 0.0000000e+00  0.0000000e+00  4.4408921e-16]]
Matriz p
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


### ¿Dado el sistema $Ax=b$, $A \in \mathbb{R}^{n \times n}$ cómo se resuelve con la factorización $PLU$?
Paso 1: encontrar factores $P,L,U$ tales que $PA=LU$.

Paso 2: resolver con el método de sustitución hacia delante el sistema triangular inferior $Ld=Pb$.

Paso 3: resolver con el método de sustitución hacia atrás el sistema triangular superior $Ux=d$.